Artificial Neural Network

Import Libraries

In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [3]:
tf.__version__

'2.16.1'

Import Dataset

In [4]:
dataset = pd.read_csv('Churn_Modelling.csv')
x = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

Encoding Categorical Data

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:,2] = le.fit_transform(x[:,2])

In [6]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One hot encoding the 'Geography' column

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [8]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


Splitting the dataset into training set and test set

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.2, random_state=0)

Feature Scaling

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

Building the ANN

Initializing the ANN

In [11]:
ann = tf.keras.models.Sequential()

Adding input layer and the first hidden layer

In [12]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Adding the second hidden layer

In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Adding the ouput layer

In [14]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Training the ANN

Compiling the ANN

In [15]:
ann.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics=['accuracy'])

Training the ANN on the Training set

In [16]:
ann.fit(x_train, y_train, batch_size = 32, epochs= 100)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 918us/step - accuracy: 0.5714 - loss: 0.6859
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - accuracy: 0.7972 - loss: 0.5469
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7981 - loss: 0.4692
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 695us/step - accuracy: 0.8085 - loss: 0.4399
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 713us/step - accuracy: 0.8080 - loss: 0.4225
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step - accuracy: 0.8247 - loss: 0.4000
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 698us/step - accuracy: 0.8267 - loss: 0.3978
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8266 - loss: 0.3930
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - accuracy: 0.8267 - loss: 0.3899
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 951us/step - accuracy: 0.8231 - loss: 0.3866
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step - accuracy: 0.8248 - loss: 0.3860
Epoch 12/100
250/250 ━━

Making the predictions and evaluating the model

Predicting the result of a single observation

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

So, should we say goodbye to that customer?

In [19]:
print(ann.predict(sc.transform([[1,0,0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
[[0.03460603]]


Predicting Test set results

In [21]:
y_pred = ann.predict(x_test)
y_pred = (y_pred>0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 935us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


Making confusiom matrix

In [23]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1502   93]
 [ 184  221]]


0.8615